In [51]:
# !wget  -O 'end_seminar.xlsx' -q 'https://www.dropbox.com/s/f4rm8sjc3v99p0m/_end_seminar.xlsx?dl=0'

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
import pandas as pd
data = pd.read_excel('end_seminar.xlsx')

In [53]:
data.head()

,timestamp,id,rating,18_group,is_mi,fall_1,fall_2,fall_3,spring_1,spring_2,spring_3,is_first_time,percentile,17_group,blended,is_ml_student
0,2020-05-15 01:12:50.543,93ff79a51cd602f1dd3028ba2c129503,698,181.0,NaN,Язык SQL,Высокопроизводительные вычисления,Матричные вычисления,Дискретная оптимизация,Численные методы,Машинное обучение 2,Да,0.183857,NaN,NaN,NaN
1,2020-05-15 02:46:48.066,26b01b1c4cd5656bab18d24c548834fb,647,181.0,NaN,Высокопроизводительные вычисления,Безопасность компьютерных систем,Язык SQL,Дискретная оптимизация,Численные методы,Машинное обучение 2,Нет,0.426009,NaN,NaN,NaN
2,2020-05-15 03:12:41.480,30f3653fc176d54e89ac3179c455c6dd,624,185.0,NaN,Безопасность компьютерных систем,Матричные вычисления,Моделирование временных рядов,Дискретная оптимизация,Машинное обучение 2,Численные методы,Да,0.538117,NaN,NaN,NaN
3,2020-05-15 04:43:08.994,1528f0eaa027580820ccf0d92a53ad68,579,182.0,NaN,Statistical Learning Theory,Высокопроизводительные вычисления,Матричные вычисления,Дискретная оптимизация,Численные методы,Машинное обучение 2,Да,0.744395,NaN,NaN,NaN
4,2020-05-15 07:47:17.197,496ea4f0d4abe264b1bb1b80eb3830c5,572,183.0,NaN,Высокопроизводительные вычисления,Безопасность компьютерных систем,Теория баз данных,Компьютерные сети,Дискретная оптимизация,Численные методы,Да,0.780269,NaN,NaN,NaN


In [54]:
data.select_dtypes(np.number)
numerical = data.select_dtypes(np.number).columns
non_numerical = pd.Index(set(data.columns) - set(numerical))
data[numerical] = data[numerical].fillna(0)
data[non_numerical] = data[non_numerical].fillna('')
data.isnull().apply(lambda col: len(col[col==True]))

timestamp        0
id               0
rating           0
18_group         0
is_mi            0
fall_1           0
fall_2           0
fall_3           0
spring_1         0
spring_2         0
spring_3         0
is_first_time    0
percentile       0
17_group         0
blended          0
is_ml_student    0
dtype: int64

In [55]:
data.sort_values(by=['timestamp'])
data = data.groupby(by='id').last()

In [56]:
data = data.sort_values(['percentile'], ascending=True)

In [57]:
courses_set = (set(data['spring_1'].values))\
| (set(data['spring_2'].values))\
| (set(data['spring_3'].values))\

courses = pd.DataFrame({'name': list(courses_set), 'max_students':np.full(len(courses_set), 30), 'students':np.full(len(courses_set), 0)}, dtype=int)
courses.loc[courses['name'] == 'Обучение с подкреплением', ['max_students']] = 60
courses.loc[courses['name'] == 'Анализ данных в бизнесе', ['max_students']] = 60
courses

,name,max_students,students
0,Компьютерное зрение,30,0
1,Промышленное программирование на языке Java,30,0
2,Компьютерные сети,30,0
3,Системы баз данных,30,0
4,Дополнительные главы прикладной статистики,30,0
5,Дискретная оптимизация,30,0
6,Обучение с подкреплением,60,0
7,Конфликты и кооперация,30,0
8,Генеративные модели в машинном обучении,30,0
9,Методы сжатия и передачи медиаданных,30,0


In [58]:
result = pd.DataFrame({'ID': data.sort_values(by=['percentile'], ascending=True).index.to_list(), 
                       'course1': np.full(len(data.index.to_list(), ), '???'),
                      'course2': np.full(len(data.index.to_list(), ), '-'),})

students_with_2_courses = data[(data['17_group'] == 171) 
                               | (data['17_group'] == 172)
                              | (data['17_group'] == 173)].index.to_list()

result.loc[result['ID'].isin(students_with_2_courses), ['course2']] = '???'

result

,ID,course1,course2
0,3a7326c9e355a67d3805824ca34c4bc5,???,???
1,3e106aaef8b24dbb01319c73e14c2936,???,-
2,e0ca1a9548af9a3f98428c62420aa484,???,-
3,7fe7df0d207babbf30249b3063ac1f26,???,-
4,86d929f6fd741e56e93cee5900298849,???,-
...,...,...,...
342,79d14db48e4737af731960ae212d5134,???,???
343,40b18d40197e3db66312a026e06b8ff1,???,-
344,be4fa2120de3397ae21eac38ff0060e3,???,-
345,d539461bf254cf8b3201cdf7990b119e,???,-


In [59]:
first_priority_courses_groupby = data.groupby('spring_1')
first_priority_courses_names = first_priority_courses_groupby.groups.keys()

second_priority_courses_groupby = data.groupby('spring_2')
second_priority_courses_names = second_priority_courses_groupby.groups.keys()

third_priority_courses_groupby = data.groupby('spring_3')
third_priority_courses_names = third_priority_courses_groupby.groups.keys()

In [60]:
for course in first_priority_courses_names:
    group = first_priority_courses_groupby.get_group(course)
    students_id = group.sort_values(by='percentile', ascending=True).index.to_list()
    for student in students_id:
        vacancies = (courses.loc[courses['name'] == course, ['max_students']].values[0] - 
                     courses.loc[courses['name'] == course, ['students']].values[0])
        if vacancies > 0:
            if result.loc[result['ID'] == student, ['course1']].values[0] == '???':
                courses.loc[courses['name'] == course, ['students']] += 1
                result.loc[result['ID'] == student, ['course1']] = course
            if result.loc[result['ID'] == student, ['course2']].values[0] == '???':
                second_course = data.loc[student, ['spring_2']].values[0]
                second_vacancies = (courses.loc[courses['name'] == second_course, ['max_students']].values[0] - 
                     courses.loc[courses['name'] == second_course, ['students']].values[0])
                if second_vacancies > 0:
                    courses.loc[courses['name'] == second_course, ['students']] += 1
                    result.loc[result['ID'] == student, ['course2']] = second_course

result

,ID,course1,course2
0,3a7326c9e355a67d3805824ca34c4bc5,Символьные вычисления,Численные методы
1,3e106aaef8b24dbb01319c73e14c2936,Дискретная оптимизация,-
2,e0ca1a9548af9a3f98428c62420aa484,Дискретная оптимизация,-
3,7fe7df0d207babbf30249b3063ac1f26,Проектирование и разработка высоконагруженных ...,-
4,86d929f6fd741e56e93cee5900298849,Дополнительные главы прикладной статистики,-
...,...,...,...
342,79d14db48e4737af731960ae212d5134,???,???
343,40b18d40197e3db66312a026e06b8ff1,???,-
344,be4fa2120de3397ae21eac38ff0060e3,???,-
345,d539461bf254cf8b3201cdf7990b119e,Обучение с подкреплением,-


In [61]:
for course in second_priority_courses_names:
    group = second_priority_courses_groupby.get_group(course)
    students_id = group.sort_values(by='percentile', ascending=True).index.to_list()
    for student in students_id:
        first_fitted = False
        vacancies = (courses.loc[courses['name'] == course, ['max_students']].values[0] - 
                     courses.loc[courses['name'] == course, ['students']].values[0])
        if vacancies > 0:
            if result.loc[result['ID'] == student, ['course1']].values[0] == '???':
                courses.loc[courses['name'] == course, ['students']] += 1
                result.loc[result['ID'] == student, ['course1']] = course
                fitted = True
            if result.loc[result['ID'] == student, ['course2']].values[0] == '???':
                if fitted == False:
                    courses.loc[courses['name'] == course, ['students']] += 1
                    result.loc[result['ID'] == student, ['course2']] = course
                elif fitted == True:
                    second_course = data.loc[student, ['spring_3']].values[0]
                    second_vacancies = (courses.loc[courses['name'] == second_course, ['max_students']].values[0] - 
                         courses.loc[courses['name'] == second_course, ['students']].values[0])
                    if second_vacancies > 0:
                        courses.loc[courses['name'] == second_course, ['students']] += 1
                        result.loc[result['ID'] == student, ['course2']] = second_course
result

,ID,course1,course2
0,3a7326c9e355a67d3805824ca34c4bc5,Символьные вычисления,Численные методы
1,3e106aaef8b24dbb01319c73e14c2936,Дискретная оптимизация,-
2,e0ca1a9548af9a3f98428c62420aa484,Дискретная оптимизация,-
3,7fe7df0d207babbf30249b3063ac1f26,Проектирование и разработка высоконагруженных ...,-
4,86d929f6fd741e56e93cee5900298849,Дополнительные главы прикладной статистики,-
...,...,...,...
342,79d14db48e4737af731960ae212d5134,???,???
343,40b18d40197e3db66312a026e06b8ff1,???,-
344,be4fa2120de3397ae21eac38ff0060e3,Численные методы,-
345,d539461bf254cf8b3201cdf7990b119e,Обучение с подкреплением,-


In [62]:
for course in third_priority_courses_names:
    group = third_priority_courses_groupby.get_group(course)
    students_id = group.sort_values(by='percentile', ascending=True).index.to_list()
    for student in students_id:
        vacancies = (courses.loc[courses['name'] == course, ['max_students']].values[0] - 
                     courses.loc[courses['name'] == course, ['students']].values[0])
        fitted = False
        if vacancies > 0:
            if result.loc[result['ID'] == student, ['course1']].values[0] == '???':
                courses.loc[courses['name'] == course, ['students']] += 1
                result.loc[result['ID'] == student, ['course1']] = course
                fitted = True
            if ((result.loc[result['ID'] == student, ['course2']].values[0] == '???') &
                (fitted == False)):
                courses.loc[courses['name'] == course, ['students']] += 1
                result.loc[result['ID'] == student, ['course2']] = course
                
result.head(347)

,ID,course1,course2
0,3a7326c9e355a67d3805824ca34c4bc5,Символьные вычисления,Численные методы
1,3e106aaef8b24dbb01319c73e14c2936,Дискретная оптимизация,-
2,e0ca1a9548af9a3f98428c62420aa484,Дискретная оптимизация,-
3,7fe7df0d207babbf30249b3063ac1f26,Проектирование и разработка высоконагруженных ...,-
4,86d929f6fd741e56e93cee5900298849,Дополнительные главы прикладной статистики,-
...,...,...,...
342,79d14db48e4737af731960ae212d5134,???,???
343,40b18d40197e3db66312a026e06b8ff1,Анализ данных в бизнесе,-
344,be4fa2120de3397ae21eac38ff0060e3,Численные методы,-
345,d539461bf254cf8b3201cdf7990b119e,Обучение с подкреплением,-


In [63]:
result.to_csv('res_spring.csv', index=None)